In [1]:
import pandas as pd
#Plotting capability
import matplotlib.pylab as plt
# Functionality for computing features
from pyspark.ml import feature

# Object for creating sequences of transformations
from pyspark.ml import Pipeline
# Load functionality to manipulate dataframes
from pyspark.sql import functions as fn

# import packages
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, LinearSVC, OneVsRest, GBTClassifier
from pyspark.ml.feature import StringIndexer

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext.setSystemProperty('spark.executor.memory', '512g')

In [4]:
data = spark.createDataFrame(pd.read_csv("Data1.csv"))

# SVM

In [5]:
feature_transformer = VectorAssembler(inputCols= ["S2_B1", "S2_B2", "S2_B4", "S2_B11", "S2_B12", "ARVI", "CI", "DVI", "GEMI", "GNDVI", "IPVI", "IRECI", "MCARI", "MSAVI", "MSAVI2", "NDVI", "NDWI2", "PSSRA", "RI", "RVI", "SAVI", "3_45_9", "3_90_12", "3_90_63", "5_90_21", "5_90_62", "7_90_15", "7_45_19", "9_180_12", "9_45_97", "9_90_11", "9_90_59", "11_135_1", "13_90_59", "15_135_97", "15_45_1", "15_45_19", "15_45_59", "15_45_81", "15_90_13", "15_90_31", "15_90_59", "15_90_64", "15_180_59"], outputCol="features")
labelIndexer = StringIndexer(inputCol="Class-label", outputCol="label").setHandleInvalid("keep")
feature_df=feature_transformer.transform(data)
data_indexed =labelIndexer.fit(feature_df).transform(feature_df)

In [6]:
training_df, validation_df, testing_df = data_indexed.randomSplit([0.8, 0.1, 0.1], seed = 100 )

In [7]:
svm_estimator = LinearSVC(featuresCol='features',labelCol='label',maxIter=5, regParam=0.01)
ovr=OneVsRest(classifier=svm_estimator,labelCol='label').fit(training_df)

In [15]:
classification_evaluator = MulticlassClassificationEvaluator(labelCol='label')
classification_evaluator_Acc = MulticlassClassificationEvaluator(labelCol='label',predictionCol="prediction", metricName="accuracy")
#classification_evaluator = MulticlassClassificationEvaluator(labelCol='label')
#accuracy=classification_evaluator.evaluate(predictions)
#print(accuracy)

In [16]:
classification_evaluator.evaluate(ovr.transform(validation_df))

0.37934904039489253

In [17]:
AUC1 = classification_evaluator.evaluate(ovr.transform(testing_df))
AUC1

0.44895632449624934

In [18]:
Accuracy1 = classification_evaluator_Acc.evaluate(ovr.transform(testing_df))
Accuracy1

0.565359477124183

In [19]:
svm_estimator1 = LinearSVC(featuresCol='features',labelCol='label',maxIter=50, regParam=0.5)
ovr1=OneVsRest(classifier=svm_estimator,labelCol='label').fit(training_df)

In [20]:
classification_evaluator.evaluate(ovr1.transform(validation_df))

0.37934904039489253

In [21]:
AUC2 = classification_evaluator.evaluate(ovr1.transform(testing_df))
AUC2

0.44895632449624934

In [22]:
Accuracy2 = classification_evaluator_Acc.evaluate(ovr1.transform(testing_df))
Accuracy2

0.565359477124183

In [23]:
svm_estimator2 = LinearSVC(featuresCol='features',labelCol='label',maxIter=100, regParam=1)
ovr2=OneVsRest(classifier=svm_estimator,labelCol='label').fit(training_df)

In [28]:
classification_evaluator.evaluate(ovr2.transform(validation_df))

0.37934904039489253

In [26]:
AUC3 = classification_evaluator.evaluate(ovr2.transform(testing_df))
AUC3

0.44895632449624934

In [27]:
Accuracy3 = classification_evaluator_Acc.evaluate(ovr2.transform(testing_df))
Accuracy3

0.565359477124183